Loading data

In [121]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB 
from sklearn.preprocessing import OrdinalEncoder
import scipy

df_inception_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\Inception_submission.csv')
df_efficient_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\EfficientNet_submission.csv')
df_resnet_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\ResNet_submission.csv')
df_vgg16_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\VGG16_submission.csv')
df_xception_old = pd.read_csv (r'C:\Users\20192236\Downloads\results\results\Xception_submission.csv')

df_inception = df_inception_old.rename(columns={'label':'label_inception'}, errors='raise')
df_efficient = df_efficient_old.rename(columns={'label':'label_efficient'}, errors='raise')
df_resnet = df_resnet_old.rename(columns={'label':'label_resnet'}, errors='raise')
df_vgg16 = df_vgg16_old.rename(columns={'label':'label_vgg16'}, errors='raise')
df_xception = df_xception_old.rename(columns={'label':'label_xception'}, errors='raise')

all_probs = pd.DataFrame(data=[df_inception['id'], df_inception['label_inception'], df_xception['label_xception'], df_efficient['label_efficient'], df_resnet['label_resnet'], df_vgg16['label_vgg16']])
all_probs = all_probs.transpose()
all_probs['mu'] = 0
all_probs['sigma'] = 0
all_probs['mu'][0]

0

Making predictions

In [123]:
mu = []
sigma = []
for i in range(len(all_probs)):
    mu.append(np.mean(all_probs.iloc[i, 1:6].values))
    sigma.append(np.std(all_probs.iloc[i, 1:6].values))


,id,label_inception,label_xception,label_efficient,label_resnet,label_vgg16,mu,sigma
0,00006537328c33e284c973d7b39d340809f7271b,0.478034,0.993316,0.503348,0.999171,0.998745,0,0
1,0000ec92553fda4ce39889f9226ace43cae3364e,0.518892,0.95245,0.959156,0.93481,0.6989,0,0
2,00024a6dee61f12f7856b0fc6be20bc7a48ba3d2,0.575155,0.835915,0.396475,0.992789,0.89388,0,0
3,000253dfaa0be9d0d100283b22284ab2f6b643f6,0.648605,0.675391,0.560131,0.984799,0.996066,0,0
4,000270442cc15af719583a8172c87cd2bd9c7746,0.192873,0.002332,0.019242,0.001395,0.01143,0,0
...,...,...,...,...,...,...,...,...
57453,fffdd1cbb1ac0800f65309f344dd15e9331e1c53,0.147148,0.000633,0.022752,0.035865,0.001352,0,0
57454,fffdf4b82ba01f9cae88b9fa45be103344d9f6e3,0.176237,0.013224,0.001911,0.048932,0.009463,0,0
57455,fffec7da56b54258038b0d382b3d55010eceb9d7,0.290546,0.000219,0.014254,0.0145,0.001723,0,0
57456,ffff276d06a9e3fffc456f2a5a7a3fd1a2d322c6,0.420927,0.503107,0.0917,0.757315,0.834874,0,0


In [96]:
def predict_Bayes_class(X,mu_list,sigma_list): 
    #Returns the predicted class from an optimal bayes classifier - distributions must be known
    scores_list = []
    classes = len(mu_list)
    
    for p in range(classes):
        score = scipy.stats.multivariate_normal.pdf(X, mean=mu_list[p], cov=sigma_list[p])
        scores_list.append(score)
             
    return np.argmax(scores_list)
predict_Bayes_class()

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [124]:
mu

[0.7945228660000001,
 0.8128417540000001,
 0.73884269,
 0.77299857,
 0.04545442966,
 0.14903411640000003,
 0.3452895165,
 0.45030844800000003,
 0.6884171208,
 0.793192514,
 0.26420837919999995,
 0.08456170464,
 0.785263782,
 0.4453313714,
 0.0358595413,
 0.6557827119999999,
 0.152138244,
 0.1419026622,
 0.183711935764,
 0.201850272,
 0.05811457498,
 0.094735227,
 0.20423416020000001,
 0.594596046,
 0.25456892700000006,
 0.0748575578,
 0.1776038458,
 0.438472836,
 0.221243017,
 0.18158329899999998,
 0.5988001512000001,
 0.22454897219999997,
 0.33698910140000005,
 0.18492252699999998,
 0.1088346738,
 0.026036106059999998,
 0.948558584,
 0.03469459379999999,
 0.5327158068,
 0.723589336,
 0.2495629794,
 0.14235754720000002,
 0.10300285940000001,
 0.48946081599999997,
 0.10633364638000001,
 0.8626956680000001,
 0.172020104,
 0.0908223028,
 0.07358540291999999,
 0.16964915900000002,
 0.837865858,
 0.047111980600000006,
 0.7162093700000001,
 0.30336045720000004,
 0.8110118780000001,
 0.578280

In [12]:
# Function that handles sample splitting, model fitting and report printing 
def mfunc(X, y, typ):
    
    # Create training and testing samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    model = typ
    clf = model.fit(X_train, y_train)

    # Predict class labels on a test data
    pred_labels = model.predict(X_test)

    # Print model attributes 
    print('Classes: ', clf.classes_) # class labels known to the classifier
    if str(typ)=='GaussianNB()':
        print('Class Priors: ',clf.class_prior_) # prior probability of each class.
    else: 
        print('Class Log Priors: ',clf.class_log_prior_) # log prior probability of each class.
        
    # Use score method to get accuracy of the model
    print('--------------------------------------------------------')
    score = model.score(X_test, y_test)
    print('Accuracy Score: ', score)
    print('--------------------------------------------------------')
    
    # Look at classification report to evaluate the model
    print(classification_report(y_test, pred_labels))
    
    # Return relevant data for chart plotting
    return X_train, X_test, y_train, y_test, clf, pred_labels

In [ ]:
# Select data for modeling
X=df[['opening_eco', 'white_id']]
y=df['white_win'].values

# Encode categorical variables
enc = OrdinalEncoder()
X = enc.fit_transform(X)

# Fit the model and print the result
X_train, X_test, y_train, y_test, clf, pred_labels = mfunc(X, y, CategoricalNB())